In [ ]:
import re

# Original string containing image URLs
image_urls = 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picQs7E2w.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/pic6WfK9K.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picuSFqBM.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picwi6J0R.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picGZdnRw.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picN5xzmG.jpg")'

# Extract URLs using regular expression
urls = re.findall(r'https?://[^\s]+', image_urls)

# Print extracted URLs
print(urls)


In [1]:
image_urls = 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picQs7E2w.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/pic6WfK9K.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picuSFqBM.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picwi6J0R.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picGZdnRw.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picN5xzmG.jpg")'

In [14]:
import re
image_urls = image_urls.replace('c(','(')

image_urls = 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picQs7E2w.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picN5xzmG.jpg")'
# image_urls_mod = ['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picQs7E2w.jpg', 'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picN5xzmG.jpg']


In [18]:
import re

# Your initial string
image_urls = 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picQs7E2w.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picN5xzmG.jpg")'

# Use regex to find all URLs within the string, ensuring it correctly excludes the final quote and anything after it
urls = re.findall(r'https?://[^\s"]+', image_urls)

# The final list of URLs, trimming the trailing characters manually if needed
image_urls_mod = [url.rstrip('",') for url in urls]  # This ensures any trailing ", or " are removed

print(image_urls_mod)

['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picQs7E2w.jpg', 'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/7/picN5xzmG.jpg']
